<a href="https://colab.research.google.com/github/paulokuriki/prompt_engeneering/blob/main/prompt_engeneering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Radiology Report Classification with Large Language Models

## Learning Objectives

By the end of this session, you will be able to:
- Understand how Large Language Models (LLMs) can assist in radiology report classification.
- Apply effective prompting techniques for medical report analysis.
- Develop specialized classifiers for different radiological findings.

---

# Introduction

Radiologists often need to classify reports based on specific findings or conditions. **Large Language Models (LLMs)** can streamline this process, improving efficiency and consistency in report classification. This tutorial demonstrates how to leverage LLMs for various classification tasks in radiology.


# Setup and Data Loading

To begin, we'll import the required libraries and load the dataset.

In [1]:
pip install langchain_openai

In [2]:
import os
import getpass
import random
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from IPython.display import HTML, display

In [3]:
# Load and prepare our dataset
def load_reports(url):
    """Load and clean radiology reports from URL"""
    df = pd.read_csv(url)
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.replace('', pd.NA).dropna().reset_index(drop=True)
    return df

# Load the data
url = "https://raw.githubusercontent.com/paulokuriki/api_for_rads/main/reports.csv"
df = load_reports(url)

# Show data distribution
print(f'Dataset downloaded successfully. Total sample reports: {len(df)}')
print(df.label.value_counts())

Dataset downloaded successfully. Total sample reports: 3927
label
abnormal    2564
normal      1363
Name: count, dtype: int64


# Report Classification Functions

To facilitate classification, we will define simple yet effective functions that help analyze radiology reports using LLMs.


In [4]:
#def get_random_report(df):
#    """Get a random report from our dataset"""
#    random.seed(40)  # Set seed for reproducibility
#    idx = random.randint(0, len(df) - 1)
#    return {
#        'report': df.iloc[idx]['report'],
#        'label': df.iloc[idx]['label'],
#        'index': idx
#    }

def classify_report(report, template, looking_for=None, examples=None):
    """
    Classify a radiology report using LLM
    """
    llm = ChatOpenAI(
        model="gpt-4-0125-preview",
        temperature=0,
        seed=42,
        model_kwargs={"response_format": {"type": "json_object"}}
    )

    # Format the template with the report and any findings
    if looking_for and examples:
        prompt = template.format(report=report, looking_for=looking_for, examples=examples)
    else:
        prompt = template.format(report=report)

    try:
        response = llm.invoke(prompt)
        result = json.loads(response.content.lower())
        return result['classification']
    except Exception as e:
        print(f"Error during classification: {e}")
        return 'error'

def classify_multiple_reports(df, n_reports=5, template=None, looking_for=None, examples=None, seed=67):
    """Classify multiple random reports"""
    results = []
    random.seed(seed)  # Set seed for reproducibility
    report_indices = random.sample(range(len(df)), n_reports)

    for idx in tqdm(report_indices, desc="Classifying reports"):
        report = df.iloc[idx]['report']
        label = df.iloc[idx]['label']
        prediction = classify_report(report, template, looking_for, examples)

        results.append({
            'index': idx,
            'report': report,
            'original_label': label,
            'predicted_label': prediction
        })

    return results

def display_results(results, show_original_label=True):
    """Display classification results in a readable format"""

    for r in results:
        print(f"\nReport #{r['index']}:")
        print("=" * 50)
        print(f"Report text:")
        display(r['report'])
        print("-" * 50)

        if show_original_label:
            # Compare prediction with the original label
            is_correct = r['original_label'].lower() == r['predicted_label'].lower()
            classification = 'Correct' if is_correct else 'Wrong'
            classification_icon = '✅' if is_correct else '👎'
            print(f"Original Label : {r['original_label']}")
            print(f"Predicted Label: {r['predicted_label']}")
            print(f"Classification : {classification} {classification_icon}")

        else:
            # Only show predicted classification when original label is hidden
            is_correct = r['predicted_label'].lower() == "present"
            classification_icon = '✓✓✓' if is_correct else 'xxx'
            classification_icon = '✅' if is_correct else '👎'
            print(f"Finding: {r['predicted_label']} {classification_icon}")

        print()


    total = len(results)
    print("SUMMARY:")
    print("=" * 50)
    print(f"\nAnalyzed {total} reports")

    # Show accuracy if original labels are available
    if show_original_label:
        correct = sum(1 for r in results if r['original_label'].lower() == r['predicted_label'].lower())

        print(f"Correct predictions: {correct}")
        print(f"Accuracy: {(correct/total)*100:.1f}%\n")


# OpenAI API Key Setup

## What is an API Key?
An API key functions like a password that grants access to OpenAI's services. It allows you to:
- Use OpenAI's models (e.g., GPT-4).
- Submit reports for analysis.
- Monitor usage and manage access securely.

## How to Obtain an OpenAI API Key:
1. Visit [OpenAI's platform](https://platform.openai.com/signup) and sign up or log in.
2. Navigate to the [API Keys page](https://platform.openai.com/api-keys).
3. Click **"Create new secret key"**.
4. Copy and securely store your API key (you won't be able to view it again!).

### Important Notes:
- **Keep your API key private** – do not share or store it in public repositories.
- **OpenAI services require payment** – ensure you have valid billing information.

In [5]:
if 'OPENAI_API_KEY' not in os.environ:
    print("Please enter your OpenAI API key (it won't be displayed as you type):")
    api_key = getpass.getpass()
    os.environ['OPENAI_API_KEY'] = api_key
    print("OpenAI API key set successfully.")
else:
    print("OpenAI API key already set.")

Please enter your OpenAI API key (it won't be displayed as you type):
··········
OpenAI API key set successfully.


# Let's Try It Out! 🏥

## 1. Zero-Shot Classification

In **zero-shot classification**, the model determines whether a given chest X-ray report describes a **normal** or **abnormal** case without prior examples.

### How It Works:
**Prompt Template:**  
- The model receives a structured instruction to classify reports.
- The response format is **JSON**, containing the key **"classification"** with possible values: `"normal"` or `"abnormal"`.
- The `{report}` placeholder is replaced with an actual chest X-ray report before passing it to the model.


In [6]:
single_shot_template = """
### INSTRUCTION
You are a specialist in chest x-ray reports. Your task is to classify if a report is normal or abnormal.
Your response should be in a JSON format with the key "classification" and the possible values: "normal" or "abnormal"

### REPORT TO CLASSIFY
{report}
"""

#for seed in range(60, 80):
results = classify_multiple_reports(df, n_reports=1, template=single_shot_template)

display_results(results, show_original_label=True)


Classifying reports: 100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


Report #306:
Report text:


'Low lung volumes. Normal heart size. The trachea is midline. Lungs are clear. No pneumothorax. No pleural effusion. No acute cardiopulmonary abnormality.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎

SUMMARY:

Analyzed 1 reports
Correct predictions: 0
Accuracy: 0.0%



### Testing Classification in Multiple Reports

In [7]:
results = classify_multiple_reports(df, n_reports=10, template=single_shot_template)

display_results(results, show_original_label=True)

Classifying reports: 100%|██████████| 10/10 [00:14<00:00,  1.49s/it]


Report #306:
Report text:


'Low lung volumes. Normal heart size. The trachea is midline. Lungs are clear. No pneumothorax. No pleural effusion. No acute cardiopulmonary abnormality.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: normal
Classification : Wrong 👎


Report #475:
Report text:


'The heart is normal in size. The mediastinum is stable. There is again significant thoracolumbar rotatory scoliosis. The aorta is atherosclerotic. The lungs are hypoinflated but clear. Hypoinflation without acute disease.'

...

--------------------------------------------------
Original Label : abnormal
Predicted Label: abnormal
Classification : Correct ✅


Report #2442:
Report text:


'Chest: The cardiomediastinal silhouette is within normal limits for size and contour. The lungs are normally inflated without evidence of focal airspace disease, pleural effusion, or pneumothorax. Thoracic spine: Mild dextro curvature the upper thoracic spine. Evaluation of the upper thoracic bodies is limited secondary to osseous overlap. Vertebral body XXXX and disc spaces are maintained. Mild degenerative endplate changes. Lumbar spine: There are 5 nonrib-bearing lumbar type vertebral bodies. Alignment is within normal limits. Vertebral body XXXX and disc spaces are maintained. Mild degenerative change without acute displaced fracture or dislocation. Moderate amount of stool.. Chest. No acute radiographic cardiopulmonary process. Thoracic spine. Mild degenerative change without acute bony abnormality. Lumbar spine. Mild degenerative change without acute bony abnormality.'

--------------------------------------------------
Original Label : normal
Predicted Label: abnormal
Classification : Wrong 👎


Report #1759:
Report text:


'There is moderate cardiomegaly. No interstitial edema or pleural effusion. No focal airspace consolidation. No pneumothorax. There is mild degenerative disc disease of the thoracic spine. 1. Cardiomegaly without radiographic evidence of heart failure. 2. No acute cardiopulmonary abnormality.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: abnormal
Classification : Correct ✅

SUMMARY:

Analyzed 10 reports
Correct predictions: 8
Accuracy: 80.0%



## 2. Few-Shot Classification

Zero-shot classification may not always be accurate. To enhance performance, we use **few-shot prompting**, where the model is provided with a few labeled examples to improve its accuracy.

### How It Works:
**Prompt Template:**  
- The `{examples}` placeholder contains relevant instances of abnormalities.
- The `{report}` placeholder is replaced with an actual chest X-ray report.
- The model is instructed to classify a report as **abnormal** if it contains specific **findings**.

In [8]:
few_show_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports.
Your task is to classify a report as abnormal if it describes signs of {looking_for}, such as:
{examples}
Consider the finding positive even if it is mild.
Your response should be in a JSON format with the key 'classification' and the possible values: 'normal' or 'abnormal'

### REPORT TO CLASSIFY
{report}
"""

looking_for='any abnormalities'

examples = """
- Low or high lung volume
- Abnormalities in the lungs, bones, heart, mediastinum or pleura
- Abnormal calcifications and/or granulomas
- Abormalities in the neck or abdomen
"""

print(f"\n{looking_for.capitalize()} Classifier")
print(f"Examples: {examples}")


results = classify_multiple_reports(df, n_reports=10,
                                    template=few_show_template,
                                    looking_for=looking_for,
                                    examples=examples)

display_results(results, show_original_label=True)


Any abnormalities Classifier
Examples: 
- Low or high lung volume
- Abnormalities in the lungs, bones, heart, mediastinum or pleura
- Abnormal calcifications and/or granulomas
- Abormalities in the neck or abdomen 



Classifying reports: 100%|██████████| 10/10 [00:12<00:00,  1.24s/it]


Report #306:
Report text:


'Low lung volumes. Normal heart size. The trachea is midline. Lungs are clear. No pneumothorax. No pleural effusion. No acute cardiopulmonary abnormality.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: abnormal
Classification : Correct ✅


Report #475:
Report text:


...

--------------------------------------------------
Original Label : abnormal
Predicted Label: abnormal
Classification : Correct ✅


Report #2442:
Report text:


'Chest: The cardiomediastinal silhouette is within normal limits for size and contour. The lungs are normally inflated without evidence of focal airspace disease, pleural effusion, or pneumothorax. Thoracic spine: Mild dextro curvature the upper thoracic spine. Evaluation of the upper thoracic bodies is limited secondary to osseous overlap. Vertebral body XXXX and disc spaces are maintained. Mild degenerative endplate changes. Lumbar spine: There are 5 nonrib-bearing lumbar type vertebral bodies. Alignment is within normal limits. Vertebral body XXXX and disc spaces are maintained. Mild degenerative change without acute displaced fracture or dislocation. Moderate amount of stool.. Chest. No acute radiographic cardiopulmonary process. Thoracic spine. Mild degenerative change without acute bony abnormality. Lumbar spine. Mild degenerative change without acute bony abnormality.'

--------------------------------------------------
Original Label : normal
Predicted Label: abnormal
Classification : Wrong 👎


Report #1759:
Report text:


'There is moderate cardiomegaly. No interstitial edema or pleural effusion. No focal airspace consolidation. No pneumothorax. There is mild degenerative disc disease of the thoracic spine. 1. Cardiomegaly without radiographic evidence of heart failure. 2. No acute cardiopulmonary abnormality.'

--------------------------------------------------
Original Label : abnormal
Predicted Label: abnormal
Classification : Correct ✅

SUMMARY:

Analyzed 10 reports
Correct predictions: 9
Accuracy: 90.0%



## 3. Condition-Specific Classification

In some cases, you may need to classify reports based on specific medical conditions rather than general abnormalities.

### How It Works:
**Prompt Template:**  
- The `{looking_for}` placeholder specifies the condition to be detected (e.g., cardiomegaly, COPD).
- The `{examples}` placeholder contains key indicators related to the condition.
- The model classifies the report as **"present"** or **"absent"** based on the findings.

### Example: Cardiomegaly Detection
- **Signs of Cardiomegaly:**
  - Increased heart size
  - Enlarged cardiac silhouette
  - Increased cardiomediastinal silhouette

In [9]:
finding_specific_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports.
Your task is to classify a report as positive if it describes signs of {looking_for}, such as:
{examples}
Consider the finding positive even if it is mild.
Your response should be in a JSON format with the key 'classification' and the possible values: 'present' or 'absent'

### REPORT TO CLASSIFY
{report}
"""

looking_for='cardiomegaly'

examples = """
- Increased heart size
- Enlarged cardiac silhouette
- Increased cardiomediastinal silhouette
"""

print(f"\n{looking_for.capitalize()} Detection")
print(f"Examples: {examples}")


results = classify_multiple_reports(df,
                                    n_reports=10,
                                    template=finding_specific_template,
                                    looking_for=looking_for,
                                    examples=examples,
                                    seed=41)

display_results(results, show_original_label=False)


Cardiomegaly Detection
Examples: 
- Increased heart size
- Enlarged cardiac silhouette
- Increased cardiomediastinal silhouette



Classifying reports: 100%|██████████| 10/10 [00:14<00:00,  1.48s/it]


Report #1560:
Report text:


'The trachea is midline. The cardio mediastinal silhouette is of normal size and contour. No evidence of focal infiltrate or effusion. Low lung volumes XXXX XXXX atelectasis and bronchovascular crowding. There is no pneumothorax. The visualized bony structures reveal no acute abnormalities. Lateral view reveals degenerative changes of the thoracic spine. 1. No acute cardiopulmonary abnormalities. 2. Low lung volumes causing bibasilar atelectasis and bronchovascular crowding .'

--------------------------------------------------
Finding: absent 👎


Report #1360:
Report text:


...

--------------------------------------------------
Finding: absent 👎


Report #3671:
Report text:


'The lungs are clear. No pleural effusion is seen. The heart and mediastinum are normal. The skeletal structures are normal. There are surgical clips in the right axilla region. No active disease.'

--------------------------------------------------
Finding: absent 👎


Report #2826:
Report text:


'Stable cardiomegaly with left-sided implanted ICD. Hilar pulmonary venous vasculature stable compared to prior examination. No focal consolidation, pneumothorax, or large pleural effusions identified. No acute osseous abnormality is identified. Stable cardiomegaly without evidence of overt heart failure/pulmonary edema.'

--------------------------------------------------
Finding: present ✅

SUMMARY:

Analyzed 10 reports


# Create Your Own Specialized Classifier! 🚀

You can create a custom classifier for any specific condition by following these steps:

1. **Define the condition** you want to detect (e.g., COPD, pleural effusion).
2. **Specify key indicators** associated with the condition.
3. **Modify the template** to reflect the new condition and findings.
4. **Run the model** to classify reports based on your selected condition.

### Example: COPD Detection
- **Signs of COPD:**
  - Hyperinflated lungs
  - Flattened diaphragm
  - Increased retrosternal airspace
  - Narrowed cardiac silhouette
  - Emphysematous changes


In [10]:
finding_specific_template = """
### INSTRUCTION
You are a specialist in chest X-ray reports.
Your task is to classify a report as positive if it describes signs of {looking_for}, such as:
{examples}
Consider the finding positive even if it is mild.
Your response should be in a JSON format with the key 'classification' and the possible values: 'present' or 'absent'

### REPORT TO CLASSIFY
{report}
"""

looking_for = 'COPD (Chronic Obstructive Pulmonary Disease)'

examples = """
- Hyperinflated lungs
- Flattened diaphragm
- Increased retrosternal airspace
- Narrowed cardiac silhouette
- Emphysematous changes
"""

print(f"\n{looking_for.capitalize()} Detection")
print(f"Examples: {examples}")


results = classify_multiple_reports(df,
                                    n_reports=10,
                                    template=finding_specific_template,
                                    looking_for=looking_for,
                                    examples=examples,
                                    seed=60)

display_results(results, show_original_label=False)


Copd (chronic obstructive pulmonary disease) Detection
Examples: 
- Hyperinflated lungs
- Flattened diaphragm
- Increased retrosternal airspace
- Narrowed cardiac silhouette
- Emphysematous changes



Classifying reports: 100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


Report #1260:
Report text:


'Normal heart size and mediastinal contours. Patchy right lower lobe airspace opacities. No pleural effusion or pneumothorax. Visualized osseous structures are unremarkable in appearance. Mild, nonconsolidating right lower lobe airspace disease. This may represent an early pneumonia.'

--------------------------------------------------
Finding: absent 👎


Report #1160:
Report text:


...

--------------------------------------------------
Finding: absent 👎


Report #1913:
Report text:


'There are chronic changes in both lungs with appearance of emphysema. No acute airspace disease. No pulmonary edema. No XXXX of pleural effusions. Findings are grossly unchanged compared with prior chest CT from XXXX.'

--------------------------------------------------
Finding: present ✅


Report #3626:
Report text:


'The cardiac silhouette size is at the upper limits of normal. Central vascular markings are mildly prominent. The lungs are normally inflated with no focal airspace disease, pleural effusion, or pneumothorax. No acute bony abnormality. 1. Mild central vascular prominence, XXXX congestion. Heart size at the upper limits of normal.'

--------------------------------------------------
Finding: absent 👎

SUMMARY:

Analyzed 10 reports


# Conclusion & Next Steps 📌

Through this notebook, we've explored how **prompt engineering** enhances the classification of radiology reports using LLMs.

### Key Takeaways:
✅ **Zero-shot prompting** enables classification without prior examples.  
✅ **Few-shot prompting** improves accuracy by providing labeled examples.  
✅ **Condition-specific classification** allows targeted detection of medical conditions.  

### What’s Next?
Now that you understand the fundamentals, here are some next steps:
1. **Test additional conditions** by modifying the `{looking_for}` and `{examples}` variables.
2. **Experiment with different LLM models** to compare performance.
3. **Refine your prompts** to improve classification accuracy.
4. **Apply real-world datasets** to validate the model in clinical settings.

By leveraging **LLMs and prompt engineering**, we can enhance efficiency in **radiology report classification**, aiding medical professionals in making informed decisions. 🏥💡

---

### Credits
This notebook is adapted from work by **Paulo Kuriki** and **Felipe Kitamura**.
